In [ ]:
from sklearn import svm
from sklearn.cross_validation import train_test_split
import numpy as np
import pandas as pd
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
import sys
from sklearn.model_selection import GridSearchCV
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    precision=TP/(TP+FP)
    recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

def svm_crossvalidation():
    if __name__=='__main__':
        datapath ='/home01/chenhuangrong/ROC/%d_interval_PSTNP.csv'%kk
        train_data = pd.read_csv(datapath, header=None, index_col=None)
        X = np.array(train_data)
        Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
        Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
        Y.extend(Y2)
        Y = np.array(Y)
        
        svc = svm.SVC()
#         parameters = {'kernel':['rbf'], 'C': np.linspace(1,50,50), 'gamma': np.linspace(0.77,0.80,100)}
#         parameters = {'kernel': ['rbf'], 'C':np.logspace(-5,15,20,base=2), 'gamma': np.logspace(-15,-5,20,base=2)}
#         parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,xrange(-2,5,1)), 'gamma':map(lambda x:2**x,xrange(-5,2,1))}
        parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,28)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,28))}
#         parameters = {'kernel': ['rbf'], 'C':np.linspace(3,5,10), 'gamma': np.linspace(0.05,0.07,10)}
        clf = GridSearchCV(svc, parameters, cv=10, n_jobs=-1, scoring='accuracy')
        clf.fit(X, Y)
        C=clf.best_params_['C']
        print clf.best_score_
        gamma=clf.best_params_['gamma']
        y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1)
        
        
        
        ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
        ACC=metrics.accuracy_score(Y,y_predict)
        precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
        F1_Score=metrics.f1_score(Y, y_predict)
        F_measure=F1_Score
        MCC=metrics.matthews_corrcoef(Y, y_predict)
        pos=TP+FN
        neg=FP+TN
        savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
        print savedata
        easy_excel.save("svm_crossvalidation",[str(X.shape[1])],savedata,'/home01/chenhuangrong/ROC/%d_interval_PSTNP_result.xls'%kk)
        
        y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
        Y=pd.DataFrame(Y)    
        y_predict_proba=pd.DataFrame(y_predict_proba)
        y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
        pd.DataFrame(y_predict_proba).to_csv('/home01/chenhuangrong/ROC/%d_interval_PSTNP_label.csv'%kk,header=None,index=False)
        
        y_predict=pd.DataFrame(y_predict)
        y_predict_=pd.concat([Y,y_predict],axis=1)
        pd.DataFrame(y_predict_).to_csv('/home01/chenhuangrong/ROC/%d_interval_PSTNP_predict.csv'%kk,header=None,index=False)
        # savafile = 'svm_crossvalidation_parameter.txt'
        # f = open(savafile, 'w+')
        # f.write("MCC:%f"%metrics.matthews_corrcoef(Y,y_predict)+'\n')
        # f.write("ACC:%f"%metrics.accuracy_score(Y,y_predict)+'\n')
        # f.write("ROC_AUC_AREA:%f"%ROC_AUC_area+'\n')
        # f.write("SN:%f,SP:%f,TP:%d:,TN:%d,FP:%d,FN:%d"%performance(Y,y_predict)+'\n')
        # f.write("ROC_AUC:%f"%metrics.roc_auc_score(Y,y_predict)+'\n')
        # f.write("C:%f,gamma:%f"%(C,gamma)+'\n')
        # f.write("F1-score:%f"%metrics.f1_score(Y,y_predict)+'\n')
        # f.write("F-measure:%f"%metrics.f1_score(Y,y_predict)+'\n')
        # f.close()
kk=1
for kk in xrange(1,6):
    svm_crossvalidation()

